In [2]:
import numpy as np
import numpy.random as npr
import cvxpy as cp

### Define Functions

In [220]:
def greedy_lower_bound(K, w, r):
    m,n = K.shape
    K_bar = 1-K
    v = np.zeros(n)
    for i in range(n):
        v[i] = np.ones(m) @ K_bar[:, i] * w[i]

    v_srtd = np.flip(np.argsort(v))
    v[v_srtd]

    # v_srtd is the arguments of the "most important" features to pick

    x = np.zeros(n)
    for i in range(n):
        x_temp = np.copy(x)
        x_temp[v_srtd[i]] = 1
        if np.count_nonzero(K @ x_temp) <= m*(1-r):
            x = x_temp

    # COUNT IS LOWER BOUND
    count = x @ w
    return count, x

In [221]:
def solve_upper_bound(K, w, r):
    m,n = K.shape
    K_bar = 1 - K
    x = cp.Variable(n, boolean = True)
    obj = cp.Maximize(w @ x)
    # cons = [cp.norm(K @ x, p = 0) <= m * (1-r)]
    cons = [r * m * cp.sum(x) - np.ones(m) @ K_bar @ x <= 0]

    prob = cp.Problem(obj, cons)
    prob.solve()
    return prob.value, x.value

In [222]:
# is the actual constraint satisfied
def true_const(K, r, x):
    m,_ = K.shape
    return np.count_nonzero(K @ x) <= m*(1-r)

## Make random data matrix

In [223]:
npr.seed(0)

m = 20
n = 10
k = 0

D = npr.randint(2, size = (m,n))
K = np.abs(D - D[k,:])
K_bar = 1-K

# information weighting vector
w = npr.randint(0, 10, size = n)
r = 0.6

In [224]:
p_val_low, x_val_low = greedy_lower_bound(K,w,r)
p_val_high, x_val_high = solve_upper_bound(K,w,r)

bound_tight = true_const(K, x_val_high,x_val_high)

In [225]:
print("Upper bound = ", p_val_high)
print("Lower_bound = ", p_val_low)
print("upper bound tight = ", true_const(K,r, x_val_high))


Upper bound =  8.0
Lower_bound =  8.0
upper bound tight =  True
